In [1]:
import sys
import array
import time
import oci
import os
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import oracledb
from langchain_community.vectorstores import oraclevs
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
from sentence_transformers import CrossEncoder
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import BaseDocumentTransformer, Document
from langchain_community.llms import OCIGenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import requests
from PyPDF2 import PdfReader
from io import BytesIO
print("Successfully imported libraries and modules")

/home/opc/.conda/envs/oig/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully imported libraries and modules


In [2]:
def chunks_to_docs_wrapper(row: dict) -> Document:
    """
    Converts text into a Document object suitable for ingestion into Oracle Vector Store.
    - row (dict): A dictionary representing a row of data with keys for 'id', 'link', and 'text'.
    """
    metadata = {'id': row['id'], 'link': row['link']}
    return Document(page_content=row['text'], metadata=metadata)

print("Successfully defined metadata wrapper")


Successfully defined metadata wrapper


In [3]:
load_dotenv()

username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
dsn = os.getenv("DB_DSN")
COMPARTMENT_OCID = os.getenv("COMPARTMENT_OCID")

print("The database user name is:", username)

try: 
    conn23c = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")


The database user name is: admin
Connection successful!


In [4]:
def load_pdf_from_url(url):
    """
    Downloads a PDF from the given URL and returns a PdfReader object.
    :param url: The URL of the PDF file.
    :return: PdfReader object.
    """
    response = requests.get(url)
    if response.status_code == 200:
        pdf_file = BytesIO(response.content)
        return PdfReader(pdf_file)
    else:
        print(f"Failed to download PDF from {url}")
        return None


In [5]:
pdf_urls = [
    # Performing Self Service Tasks with Oracle Identity Governance
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/oigdk/reference-oracle-identity-governance-docker-and-kubernetes.pdf",

    # https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/oigdk/reference-oracle-identity-governance-docker-and-kubernetes.pdf
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omadm/administering-oracle-identity-governance.pdf",

    # Developing and Customizing Applications for Oracle Identity Governance
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omdev/developing-and-customizing-applications-oracle-identity-governance.pdf",

    # Release Notes for Oracle Identity Management
    "https://docs.oracle.com/en/middleware/idm/suite/12.2.1.4/idmrn/release-notes-oracle-identity-management.pdf",

    # Upgrading Oracle Identity Manager
    "https://docs.oracle.com/en/middleware/fusion-middleware/12.2.1.4/iamup/upgrading-oracle-identity-manager.pdf",

    # Performing Self Service Tasks with Oracle Identity Governance
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omusg/performing-self-service-tasks-oracle-identity-governance.pdf",

    # Help Topics for Oracle Identity Governance
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omhlp/help-topics-oracle-identity-governance.pdf"

]

pdf_readers = [load_pdf_from_url(url) for url in pdf_urls]

# Check if all PDFs were loaded successfully
for i, pdf in enumerate(pdf_readers):
    if pdf:
        print(f"PDF {i+1} loaded successfully with {len(pdf.pages)} pages.")
    else:
        print(f"PDF {i+1} failed to load.")


PDF 1 loaded successfully with 5 pages.
PDF 2 loaded successfully with 719 pages.
PDF 3 loaded successfully with 810 pages.
PDF 4 loaded successfully with 83 pages.
PDF 5 loaded successfully with 281 pages.
PDF 6 loaded successfully with 486 pages.
PDF 7 loaded successfully with 325 pages.


In [6]:
for pdf in pdf_readers:
    if pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
        # Process the text as needed
        print(f"Processed PDF with {len(pdf.pages)} pages.")


Processed PDF with 5 pages.
Processed PDF with 719 pages.
Processed PDF with 810 pages.
Processed PDF with 83 pages.
Processed PDF with 281 pages.
Processed PDF with 486 pages.
Processed PDF with 325 pages.


In [7]:
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=800, chunk_overlap=100, length_function=len)
chunks = text_splitter.split_text(text)

In [8]:
docs = [chunks_to_docs_wrapper({'id': page_num, 'link': f'Page {page_num}', 'text': text}) for page_num, text in enumerate(chunks)]
print("Created metadata wrapper with the chunks")

Created metadata wrapper with the chunks


In [10]:
# Initialize embedding model
model_4db = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Vectorize and store chunks in Oracle Database 23ai
s1time = time.time()
knowledge_base = OracleVS.from_documents(docs, model_4db, client=conn23c, table_name="OIG_v1_table", distance_strategy=DistanceStrategy.DOT_PRODUCT)
s2time = time.time()

print(f"Vectorizing and inserting chunks duration: {round(s2time - s1time, 1)} sec.")


2024-08-13 20:27:12,856 - ERROR - An unexpected error occurred: DPY-4011: the database or network closed the connection
Help: https://python-oracledb.readthedocs.io/en/latest/user_guide/troubleshooting.html#dpy-4011
Traceback (most recent call last):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 54, in wrapper
    return func(*args, **kwargs)
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 427, in drop_table_purge
    if _table_exists(client, table_name):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 84, in _table_exists
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/oracledb-2.4.0b1-py3.9-linux-aarch64.egg/oracledb/cursor.py", line 701, in execute
    impl.execute(self)
  File "src/oracledb/impl/thin/cursor.pyx", li

In [9]:
batch_size = 100 
conn23c = oracledb.connect(user=username, password=password, dsn=dsn)
print("connection successful")
print(oraclevs._compare_version("2.4.0b1", "2.3.0"))
# Initialize embedding model
model_4db = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
for i in range(0, len(docs), batch_size):
    batch_docs = docs[i:i + batch_size]
    s1time = time.time()
    OracleVS.from_documents(batch_docs, model_4db, client=conn23c, table_name="OIG_base_table", distance_strategy=DistanceStrategy.DOT_PRODUCT)
    s2time = time.time()
    print(f"Processed batch {i//batch_size + 1} in {round(s2time - s1time, 1)} sec.")


connection successful
False
Processed batch 1 in 15.7 sec.
Processed batch 2 in 13.0 sec.
Processed batch 3 in 12.5 sec.
Processed batch 4 in 12.4 sec.
Processed batch 5 in 12.3 sec.
Processed batch 6 in 11.2 sec.
Processed batch 7 in 12.2 sec.
Processed batch 8 in 2.2 sec.


In [23]:
batch_size = 100 
conn23c = oracledb.connect(user=username, password=password, dsn=dsn)
print("connection successful")
print(oraclevs._compare_version("2.4.0b1", "2.3.0"))
knowledge_base = []
# Initialize embedding model
model_4db = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
for i in range(0, len(docs), batch_size):
    batch_docs = docs[i:i + batch_size]
    s1time = time.time()
    current_knowledge = OracleVS.from_documents(batch_docs, model_4db, client=conn23c, table_name="OIG_base_table", distance_strategy=DistanceStrategy.DOT_PRODUCT)
    knowledge_base.append(current_knowledge)
    s2time = time.time()
    print(f"Processed batch {i//batch_size + 1} in {round(s2time - s1time, 1)} sec.")


connection successful
False
Processed batch 1 in 15.7 sec.
Processed batch 2 in 13.3 sec.
Processed batch 3 in 12.4 sec.
Processed batch 4 in 12.8 sec.
Processed batch 5 in 12.6 sec.
Processed batch 6 in 11.9 sec.
Processed batch 7 in 12.4 sec.
Processed batch 8 in 2.2 sec.


In [26]:
# All at once
batch_size = 100 
conn23c = oracledb.connect(user=username, password=password, dsn=dsn)
print("connection successful")
# Initialize embedding model
model_4db = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
s1time = time.time()
vector_stores = OracleVS.from_documents(docs, model_4db, client=conn23c, table_name="OIG_base_table", distance_strategy=DistanceStrategy.DOT_PRODUCT)
s2time = time.time()
print(f"Processed batch {i//batch_size + 1} in {round(s2time - s1time, 1)} sec.")

connection successful
Processed batch 8 in 89.4 sec.


In [29]:
user_question = "What are the key features of Oracle Identity Governance?"
print("The prompt to the LLM will be:", user_question)


The prompt to the LLM will be: What are the key features of Oracle Identity Governance?


In [ ]:
print(len(docs))
if conn23c is None:
    print("Database connection is closed. Reconnecting...")
conn23c = oracledb.connect(user=username, password=password, dsn=dsn)


In [ ]:
batch_size = 100 
conn23c = oracledb.connect(user=username, password=password, dsn=dsn)
print("connection successful")
print(oraclevs._compare_version("2.4.0b1", "2.3.0"))

In [ ]:
print(oraclevs._compare_version("2.4.0b1", "2.3.0"))

In [18]:
# Configuration Profile
config_profile = "DEFAULT"
config = oci.config.from_file('~/.oci/config', config_profile)

# Define the compartment and service endpoint
# real compartment id: ocid1.user.oc1..aaaaaaaavprlus6kzst2a76z74hqfcdollr6ielt66mo75w5zwld4byfnptq
compartment_id = "ocid1.compartment.oc1..aaaaaaaagvtl2poaovkpbtndd4fzi4ro5qx3bwcwlqzmqcgcjuzujelrjq2q"
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

# Define the model details, using the model ID from OCI
model_id = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyawk6mgunzodenakhkuwxanvt6wo3jcpf72ln52dymk4wq"

# Instantiate the OCI GenAI LLM with Cohere Command Model
llm = OCIGenAI(
    model_id=model_id,
    service_endpoint=endpoint,
    compartment_id=compartment_id,
    model_kwargs={
        "temperature": 0.7, 
        "top_p": 0.75, 
        "max_tokens": 2000,
        "frequency_penalty": 0,
        "top_k": 0
    },
    auth_type="API_KEY"
)

print("LLM setup complete.")

LLM setup complete.


In [14]:
# Default config file and profile
config = oci.config.from_file('~/.oci/config', config_profile)

# validate the default config file
oci.config.validate_config(config)

In [32]:
# Setup basic variables
# Auth Config
# Update with your own compartment ID and profile if needed
compartment_id = "ocid1.compartment.oc1..aaaaaaaagvtl2poaovkpbtndd4fzi4ro5qx3bwcwlqzmqcgcjuzujelrjq2q"
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)
# Service endpoint
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
# Create the Generative AI Inference Client
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10, 240)
)
# Set up the chat request with Cohere Command model
def get_llm_response(input_text: str) -> str:
    chat_detail = oci.generative_ai_inference.models.ChatDetails()
    chat_request = oci.generative_ai_inference.models.CohereChatRequest()
    chat_request.message = input_text
    chat_request.max_tokens = 600
    chat_request.temperature = 1
    chat_request.frequency_penalty = 0
    chat_request.top_p = 0.75
    chat_request.top_k = 0
    # Specify the model ID for the Cohere Command model
    chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(
        model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyawk6mgunzodenakhkuwxanvt6wo3jcpf72ln52dymk4wq"
    )
    chat_detail.chat_request = chat_request
    chat_detail.compartment_id = compartment_id
    # Get the response from the LLM
    chat_response = generative_ai_inference_client.chat(chat_detail)
    # Extract the response text

    # print("**************************Chat Response Attributes**************************")
    # print(dir(chat_response))
    # print("**************************Chat Response Data**************************")
    # print(chat_response.data)

    # response_text = chat_response.data.text.strip() if hasattr(chat_response.data, 'text') else str(chat_response.data)

    response_text = chat_response.data.chat_response.text

    return response_text

# Example usage:
input_question = "List the features in Oracle Identity Governance"
response = get_llm_response(input_question)
print("**************************Chat Result**************************")
print(response)


**************************Chat Result**************************
Oracle Identity Governance (OIG) Suite is a comprehensive product that governs and manages an organization's digital identities and provides single sign-on (SSO) capabilities. Here are some of the key features of Oracle Identity Governance:

1. Oracle Access Manager: Enables users to secure and manage access to enterprise applications, web services, and resources with SSO capabilities. It also provides comprehensive authentication and authorization policies.

2. Oracle Identity Manager (OIM): Helps in managing the identity life cycle of users, including user provisioning, de-provisioning, and password synchronization. It also enables self-service capabilities for users to update their profiles and manage their access requests.

3. Oracle Identity Analytics: Provides advanced analytics and reporting capabilities to identify potential security threats, comply with regulatory requirements, and optimize identity governance pro

In [27]:
# Set up the template for the questions and context, and instantiate the database retriever object
template = """Answer the question based only on the  following context:
            {context} Question: {question} """
prompt = PromptTemplate.from_template(template)
retriever = vector_stores.as_retriever()

In [41]:
# RAG Steps 6 and 7 Chain the entire process together, retrieve the context, construct the prompt with the question and context, and pass to LLM for the response

s5time = time.time()
print("We are sending the prompt and RAG context to the LLM, wait a few seconds for the response...")
chain = (
  {"context": retriever, "question": RunnablePassthrough()}
     | prompt
     | (lambda x: str(x))
     | get_llm_response
     | StrOutputParser()
)
response = chain.invoke(user_question)
print(user_question)
print(prompt)
print(response)
# Print timings for the RAG execution steps

s6time = time.time()
print("")
print( f"Send user question and ranked chunks to LLM and get answer duration: {round(s6time - s5time, 1)} sec.")

We are sending the prompt and RAG context to the LLM, wait a few seconds for the response...
What are the key features of Oracle Identity Governance?
input_variables=['context', 'question'] template='Answer the question based only on the  following context:\n            {context} Question: {question} '
Based on the provided context, here are some key features of Oracle Identity Governance:

1. Self-service operations: Users can perform tasks like requesting roles and resources and managing approvals, provisioning, and certification processes without requiring administrative intervention.

2. Delegated Administration: Oracle Identity Governance allows for delegated administrators who can carry out identity administration tasks and establish authorization policies. These administrators can manage various aspects, including user roles, access privileges, and workflow rules.

3. Accessibility: Oracle Identity Governance aims to be accessible to users with disabilities. It provides accessib

In [34]:
# Save the model configuration
model_name = "sentence-transformers/all-MiniLM-L6-v2"
# Re-initialize the model later
model_4db = HuggingFaceEmbeddings(model_name=model_name)

In [35]:
# Reconnect to Oracle Database
conn23c = oracledb.connect(user=username, password=password, dsn=dsn)
# Reinitialize the vector store (replace 'OIG_base_table' with your table name)
knowledge_base = OracleVS(
    client=conn23c,
    embedding_function=model_4db,
    table_name="OIG_base_table",
    distance_strategy=DistanceStrategy.DOT_PRODUCT,
)

In [39]:
import pickle
# Save only the serializable components
serializable_chain = (
  {"context": retriever, "question": RunnablePassthrough()}
     | prompt
     | (lambda x: str(x))
     | get_llm_response
     | StrOutputParser()
)
# Save configurations and serializable chain
config_to_save = {
    'model_name': model_name,
    'table_name': "OIG_base_table", 
    'chain': serializable_chain
}
with open('rag_chain_config.pkl', 'wb') as f:
    pickle.dump(config_to_save, f)

ERROR - Exception
Traceback (most recent call last):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2773883/1441426183.py", line 17, in <module>
    pickle.dump(config_to_save, f)
  File "<stringsource>", line 2, in oracledb.thin_impl.ReadBuffer.__reduce_cython__
TypeError: no default __reduce__ due to non-trivial __cinit__
TypeError: no default __reduce__ due to non-trivial __cinit__

In [44]:
sample_question = "Can you tell me how to provide information to flat files?"
chain.invoke(sample_question)

'To provide information to flat files, you first need to specify the flat file location (flatFileLocation). Clicking Parse Headers will list all the field names present in the flat file. You can then use the Flat File Schema Properties section to create the flat file schema. Within this section, you can define options for the flat file fields, including adding new fields, selecting attributes, choosing data types, and mapping to names or UIDs. Additionally, you may need to fill in the Reconciliation Jobs section under the Settings tab, ensuring that mandatory parameters are entered.'

In [45]:
sample_question = "Can you tell me how to provide information to flat files?"
start_time = time.time()
print(chain.invoke(sample_question))
end_time = time.time()
print("")
print( f"Time it took: {round(end_time - start_time, 1)} sec.")

To provide information to flat files, you first need to specify the flat file location (flatFileLocation). Click 'Parse Headers' to list all the field names present in the flat file. This will bring up the Flat File Schema Properties section, where you can input the information you wish to include. You can add new fields, as well as edit attributes, data types, names, and UIDs.

Time it took: 1.6 sec.


In [47]:
sample_question = "Can you tell me about Setting Up Database in Server-Authentication SSL Mode"
start_time = time.time()
print(chain.invoke(sample_question))
end_time = time.time()
print("")
print( f"Time it took: {round(end_time - start_time, 1)} sec.")

Sorry, I couldn't find any information about Setting Up Database in Server-Authentication SSL Mode from the provided text. The text primarily focuses on Oracle Identity Manager and Self Service features, including unauthenticated self-service tasks, user registration, and identity administration. If you have additional context or questions related to setting up databases in Server-Authentication SSL Mode, please provide them, and I'd be happy to assist further.

Time it took: 1.7 sec.


In [48]:
# https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omadm/administering-oracle-identity-governance.pdf
# Page 157
sample_question = "Can you tell me about Configuring Password Policies for Application Instances?"
start_time = time.time()
print(chain.invoke(sample_question))
end_time = time.time()
print("")
print( f"Time it took: {round(end_time - start_time, 1)} sec.")

I can certainly help you with that. 

 Configuring password policies can be done using the following steps:
1. Go to the Password Policies section
2. Enter a name for your policy 
3. Add a description 
4. Set the minimum password age in days 
5. Determine the number of days a user is notified about their password expiration, also known as Warn After (Days)
6. Specify how often users can reuse old passwords
7. Set the maximum password duration in days
8. Evaluate passwords against complex criteria, ensuring they adhere to any requirements. If you enable the Complex Password option, you won't be able to use Custom Policy. 
9. However, if you select Custom Policy, you can set a custom password policy using fields like maximum password length, maximum repeated characters, minimum numeric characters, and minimum alphanumeric characters. 

Make sure you detatch the Policies page into a separate window so you can refer to it as you follow these steps.

Time it took: 3.6 sec.


In [49]:
sample_question = "What is your knowledge base comprised of?"
start_time = time.time()
print(chain.invoke(sample_question))
end_time = time.time()
print("")
print( f"Time it took: {round(end_time - start_time, 1)} sec.")

My knowledge base is comprised of the following:
- Users with Any Level of Risk
- Users with High Risk Summaries
- Users with High Risk Roles
- Users with High Risk Application Instances
- Only User with High Risk Entitlements

It also includes the options available to specify constraints to the base selection for the following certification definitions:
- Role certification definition
- Application instance certification definition
- Entitlement certification definition

Additionally, there is some text which appears to be an excerpt from a book or manual, referring to Oracle programs, documentation, and data, and their use by US Government end users. This text discusses the categories of "commercial computer software," "commercial computer software documentation," and "limited rights data," and the regulations that apply to them. Lastly, there is a section about IT resources, including instructions for creating, editing, deleting, and resetting them, as well as accessing resource det

In [50]:
sample_question = "What are some example roles I can give to my users as an admin?"
start_time = time.time()
print(chain.invoke(sample_question))
end_time = time.time()
print("")
print( f"Time it took: {round(end_time - start_time, 1)} sec.")

2024-08-14 22:20:06,476 - ERROR - An unexpected error occurred: DPY-4011: the database or network closed the connection
Help: https://python-oracledb.readthedocs.io/en/latest/user_guide/troubleshooting.html#dpy-4011
Traceback (most recent call last):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 54, in wrapper
    return func(*args, **kwargs)
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 776, in similarity_search_by_vector_with_relevance_scores
    cursor.execute(query, embedding=embedding_arr)
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/oracledb-2.4.0b1-py3.9-linux-aarch64.egg/oracledb/cursor.py", line 701, in execute
    impl.execute(self)
  File "src/oracledb/impl/thin/cursor.pyx", line 178, in oracledb.thin_impl.ThinCursorImpl.execute
  File "src/oracledb/impl/thin/protocol.pyx", line 440, in oracledb.thin_impl.Protocol._process_s